<a href="https://colab.research.google.com/github/stupidbutsmart/DELE-CA1/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section A CA1 CNN

Name: Jovan Heng Ghim Hong

Class: DAAA/2B/22

Admin No: 2401418

In [34]:
# Basic Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten , Conv2D
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing import image

## for keras classifier i will be using scikeras lib
## 2.12+ no longer supports tensorflow.keras.wrappers.scikit_learn
## this was coded in colab, thus 2.19 is used
from scikeras.wrappers import KerasClassifier


warnings.filterwarnings('ignore');

### Creating UDFs to aid workflow

Before i start, i want to create some functions that can help me with my work.

basic functions like:
1. create model function
2. autoencoder function
3. data processing functions

etc.

This can help modularize my code (especially since im working with 2 models) and make it easier for future collaborater to understand

In [35]:
DATASET_PATH = './datasets/'

def get_data(data_type):
  data = pd.DataFrame(columns = ['sequence'])

  for target in os.listdir(DATASET_PATH + data_type):
    # create col for target
    data[target] = 0
    for file_name in os.listdir(DATASET_PATH + data_type + target):
      img = image.load_img(DATASET_PATH + data_type + target + file_name)
      img = image.img_to_array(img)
      data.append({ 'sequence': img , target: 1})

  return data



In [32]:
def create_model(
    number_of_layers=3 , number_of_neurons=128,
    optimizer='adam' , activation='relu',
    layers=[Conv2D], # incase i want to add more layers
    strict_follow_layers = False,
    input_shape=(23 , 23 , 3) # follows first layer
):
  model = Sequential()

  # Adding layers
  if strict_follow_layers:
    for layer in layers:
      model.add(layer(number_of_neurons , activation=activation))
  else:
    for i in range(number_of_layers):
      for layer in layers:
        model.add(layer(number_of_neurons , activation=activation))

  model.add(Flatten())
  model.add(Dense(1 , activation='softmax'))
  model.compile(optimizer=optimizer , loss='categorical_crossentropy' , metrics=['accuracy'])

  return KerasClassifier(model)

